In [ ]:
import csv
import time
import string
import numpy as np
import pandas as pd
import torch as pt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython.display import clear_output
from transformers import GPT2ForSequenceClassification, ReformerModelWithLMHead, get_linear_schedule_with_warmup
from pytorch_transformers import GPT2Tokenizer
from Learning import *
device = pt.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")

In [ ]:
cats, cats_sing, phrases = Listset().load()  # Import word lists dataset

In [ ]:
# Since we do not use any of the below characters (commas and fullstops always preceded by a word), we do not have to worry
# about dividing our token-specific probability among the various possible token encodings
tkn_map = {i: tokenizer.decode(i) for i in range(len(tokenizer))}
tkn_invmap = {w: [i for i in tkn_map if tkn_map[i] == w] for w in set(tkn_map.values())}
[(k, tkn_invmap[k]) for k in tkn_invmap if len(tkn_invmap[k]) > 1]

In [ ]:
   ###   Options   ###

model_name = "ernst_one"             # First, use a mean pooling of the claim and chained evidence
test_set_frac = 0.25                 # Fraction of samples to keep as separate test set  (list sets)
sample_test_n = 25                   # Number of randomly generated prompts for each sample when testing model
log_period_batches = 25              # Batches per iteration
learning_rate = 5e-5                 # Adam learning rate (default is 5e-5, sentiment classification example had 2e-5)
adam_epsilon = 1e-8                  # Adam epsilon (default is 1e-8)
n_sched_warmup = 0                   # Linear scheduler for optimizer number of warmup steps
batch_size = bsz = 8                 # Samples per batch
N_train_batches = int(1e7 / bsz)     # Total number of batches to show model
min_nw, max_nw = 0.17, 0.8           # Minimum and maximum fraction of list to keep when truncating
max_listlen = 20                     # Maximum number of words in the list when creating a prompt (at least prior to * max_nw)
lidstone_e = 0.0                     # Smoothing for possible words/subwords which are not in the missing list words set

In [ ]:
lprompts_encoded = [(tokenizer.encode(prompt), "types of" in prompt) for prompt in lprompts]
cats_e = [[tokenizer.encode(c + ': ') for c in cs] for cs in cats]
cats_sing_e = [[tokenizer.encode(c + ': ') for c in cs] for cs in cats_sing]
phrases_e = [[tokenizer.encode(p + ', ') for p in ps] for ps in phrases]
N_tokens = len(tokenizer)
N_wordlists = len(cats)

In [ ]:
# Generate a fixed test set and save to disk, using nw_draw = 15. This function defines the next list token prediction problem
def gen_truncated_list(prmt, p):  # prmt = prompt tokens, p = list phrases tokens list
    tkzs, sent, tkix = [], [], 0
    incl_words = np.random.choice(len(p), min(max_listlen, len(p)), replace=False)
    for phz_i in incl_words:
        phz_enc = p[phz_i]
        tkzs.append((tkix, phz_enc))
        tkix += len(phz_enc)
        sent += phz_enc
    missing_w = [p[i] for i in range(len(p)) if i not in incl_words]
    trunc_ix = np.random.randint(round(tkix * min_nw), round(tkix * max_nw))
    trunc_n = min([(trunc_ix - ix) for (ix, enc) in tkzs if ix <= trunc_ix])  # N. end phrase tokens
    missing_w += [enc for (ix, enc) in tkzs if ix >= (trunc_ix - trunc_n)]
    missing_matches = missing_w
    if trunc_n > 0:
        phr_start = trunc_ix - trunc_n
        partial_phr = sent[phr_start:trunc_ix]
        missing_matches = [enc for enc in missing_w if enc[:trunc_n] == partial_phr]
    next_tokens = [enc[trunc_n] for enc in missing_matches]
    norm = len(next_tokens) * (1.0 + lidstone_e)
    tunit, y_ = 1 / norm, np.tile(lidstone_e / N_tokens, N_tokens)
    for token in next_tokens: y_[token] += tunit
    return np.hstack([prmt, sent[:trunc_ix]]), y_
def gen_samples_uniform(xcp, xcs, xp, nw, verbose=False):  # Weight testing samples (word lists) uniformly
    xs, ys, sqlens, j = [], [], [], 0
    for i in range(len(xcp)):
        x, y, sqlen = [], [], []
        cp, cs, p = xcp[i], xcs[i], xp[i]
        for m in range(nw):
            prmt, typesof = lprompts_encoded[np.random.randint(len(lprompts_encoded))]
            cat_ix = np.random.randint(len(cp))
            x_, y_ = gen_truncated_list(np.hstack([prmt, cp[cat_ix] if typesof else cs[cat_ix]]), p)
            x.append(x_)
            y.append(y_)
            sqlen.append(len(x_))
            j += 1
            if verbose and j % 100 == 0:
                sys_print("\rDone: " + str(j))
        xs.append(x)
        ys.append(y)
        sqlens.append(sqlen)
    if verbose: sys_print("\rDone: " + str(j) + ", finished!\n")
    return xs, ys, sqlens
def gen_samples(xcp, xcs, xp, n):  # Maximise training batch diversity by randomly sampling the word lists
    xs, ys, sqlens, j = [], [], [], 0
    n_sets = len(xcp)
    for m in range(n):
        i = np.random.randint(n_sets)
        cp, cs, p = xcp[i], xcs[i], xp[i]
        prmt, typesof = lprompts_encoded[np.random.randint(len(lprompts_encoded))]
        cat_ix = np.random.randint(len(cp))
        x_, y_ = gen_truncated_list(np.hstack([prmt, cp[cat_ix] if typesof else cs[cat_ix]]), p)
        xs.append(x_)
        ys.append(y_)
        sqlens.append(len(x_))
    return xs, ys, sqlens

In [ ]:
N_test = int(test_set_frac * N_wordlists)
N_train = N_wordlists - N_test
# test_idx = np.random.choice(N_wordlists, N_test, replace=False)
test_idx = np.array([3, 7])
cats_e_test, cats_sing_e_test = [cats_e[i] for i in test_idx], [cats_sing_e[i] for i in test_idx]
phrases_e_test = [phrases_e[i] for i in test_idx]
train_idx = [i for i in range(N_wordlists) if i not in test_idx]
cats_e_train, cats_sing_e_train = [cats_e[i] for i in train_idx], [cats_sing_e[i] for i in train_idx]
phrases_e_train = [phrases_e[i] for i in train_idx]
test_cats = [cats[i][0] for i in test_idx]
test_xs, test_ys, test_sqlens = gen_samples_uniform(cats_e_test, cats_sing_e_test, phrases_e_test, sample_test_n, verbose=True)

In [ ]:
save_ld((test_xs, test_ys, test_sqlens), "test.data")

In [ ]:
test_xs, test_ys, test_sqlens = load_ld("test.data")

In [ ]:
# Define next batch function
def adapt_form(xs, ys, sqlens):
    max_len = max(sqlens)
    xs = pt.tensor(np.vstack([np.pad(x, (0, max_len - len(x)), constant_values=50256) for x in xs]))
    ys = pt.tensor(np.vstack(ys))
    sqlens = pt.tensor(np.asarray(sqlens))
    return xs, ys, sqlens
def next_batch(sz):
    global cats_e_train, cats_sing_e_train, phrases_e_train
    xs, ys, sqlens = gen_samples(cats_e_train, cats_sing_e_train, phrases_e_train, sz)
    return adapt_form(xs, ys, sqlens)

In [ ]:
print(gc.collect())
create_folder("models")
create_folder("models/pretrained")
create_folder("models/pretrained/GPT2SeqClas")
model = GPT2ForSequenceClassification.from_pretrained('gpt2-large',
    output_hidden_states=True, output_attentions=True, 
    cache_dir="models/pretrained/GPT2SeqClas")
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id
model.to(device)
print("GPT2 loaded")
gc.collect()

In [ ]:
llayer = nn.Linear(model.config.n_embd, len(tokenizer), bias=False)
# softmax = nn.Softmax()
bcewl_loss = nn.BCEWithLogitsLoss()
# nll_loss = nn.NLLLoss()
# kl_loss = nn.KLDivLoss()
optimizer = pt.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=n_sched_warmup, num_training_steps=N_train_batches)
def train_step():
    global model, llayer, bcewl_loss, optimizer, scheduler
    x_batch, y_batch, sqlens_batch = next_batch(batch_size)
    x_batch, y_batch, sqlens_batch = x_batch.to(device), y_batch.to(device), sqlens_batch.to(device)
    
    model.zero_grad()
    outputs = model(x_batch.long())
    out_idx = pt.unsqueeze(pt.unsqueeze(sqlens_batch - 1, 1).repeat((1, model.config.n_embd)), 1).type(pt.int64)
    outs = pt.gather(outputs[2][-1], 1, out_idx).squeeze(1)
    logits = llayer(outs)
    
#     logsofts = pt.log(softmax(logits))
    loss = bcewl_loss(logits, y_batch.float())
    correct = pt.mean((y_batch[pt.arange(batch_size), pt.argmax(logits, axis=1)] > (lidstone_e / N_tokens)).float())
    loss_, correct_ = loss.detach().cpu().numpy(), correct.detach().cpu().numpy()
    
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    return loss_, correct_

def inference(x, sqlens):
    global model, llayer
    
    x, sqlens = x.to(device), sqlens.to(device)
    outputs = model(x.long())
    out_idx = pt.unsqueeze(pt.unsqueeze(sqlens - 1, 1).repeat((1, model.config.n_embd)), 1).type(pt.int64)
    outs = pt.gather(outputs[2][-1], 1, out_idx).squeeze(1)
    logits = llayer(outs)
    return logits
def eval_test(x, y, sqlens):
    global bcewl_loss
    
    with pt.no_grad():
        logits = inference(x, sqlens)
        loss = bcewl_loss(logits, y.float())
        correct = pt.mean((y[pt.arange(x.shape[0]), pt.argmax(logits, axis=1)] > (lidstone_e / N_tokens)).float())
        loss_, correct_ = loss.numpy(), correct.numpy()
    return loss_, correct_

# top_next = [self.tokenizer.decode(i.item()).strip() for i in probs.topk(k)[1]]

In [ ]:
batch_i = 0
best_acc, best_loss = 0, np.inf
best_acc_idx = -1
create_folder("models")
create_folder("model_logs")
create_folder("models/" + model_name)
graphs_folder = "graphs"
create_folder(graphs_folder)
train_loss, train_accuracy, test_loss, test_accuracy = [], [], [], []